# API Requests and Endpoints

#### The notebook initialization

In [1]:
from IPython.display import HTML
from init import *

## Endpoint: /

In [2]:
HTML(requests.get(server).text)

## Prometheus metrics

In [3]:
requests.get(f'{server}/metrics').text.split('\n')

['# TYPE Run counter',
 'mc_Run 9',
 '# TYPE get_functions counter',
 'mc_get_functions 2',
 '# TYPE get_metrics counter',
 'mc_get_metrics 2',
 '# TYPE model counter',
 'mc_model 9',
 '# TYPE mc_Run summary',
 'mc_Run_count 9',
 'mc_Run_seconds_sum 3.893007',
 'mc_Run_seconds_mean 0.43255633',
 'mc_Run_seconds_stddev 0.46642867',
 '# TYPE mc_get_functions summary',
 'mc_get_functions_count 2',
 'mc_get_functions_seconds_sum 0.000152',
 'mc_get_functions_seconds_mean 7.6e-05',
 'mc_get_functions_seconds_stddev 1.4e-05',
 '# TYPE mc_get_metrics summary',
 'mc_get_metrics_count 1',
 'mc_get_metrics_seconds_sum 7.3e-05',
 'mc_get_metrics_seconds_mean 7.3e-05',
 'mc_get_metrics_seconds_stddev 0',
 '# TYPE mc_model summary',
 'mc_model_count 9',
 'mc_model_seconds_sum 3.8954690000000003',
 'mc_model_seconds_mean 0.4328299',
 'mc_model_seconds_stddev 0.46647245',
 'mc_process_residential_memory_kB 22790508',
 '']

## Model

In [4]:
model = {'TimeStart': 0,
 'TimeSteps': 1000,
 'NumPaths': 100000,
 'updaters': [
     {'name': 'BrownianMotion', 'start': [0.1], 'args': [0.2,2]}
 ],
 'evaluations': [{'state': 0, 'time': 2}]
}
model_json = json.dumps(model)
model_json

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "BrownianMotion", "start": [0.1], "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2}]}'

In [5]:
requests.post(f'{server}/model',model_json).json()

{'histograms': [],
 'mean': [0.5016462206840515],
 'names': ['BrownianMotion'],
 'npaths': [100000],
 'skewness': [0.0007283107261173427],
 'stddev': [2.8189284801483154],
 'time_points': [2.0],
 'time_steps': [999]}

## Endpoint: "/functions"

In [6]:
functions = requests.get(f'{server}/functions').json()
functions

[{'doc_md': "\n# Barrier\n\nA barrier state starts from some initial value S0.\nOn each computation step we verify the barrier crossing condition, and if it is satisfied, the barrier state will\nbe set to STATE=valueOnCrossing.\n\nFor direction 'up':\n\n    if(Sold<level and Snew>=level)\n        barrierState = valueOnCrossing;\n\nFor direction 'down':\n\n    if(Sold>level and Snew<=level)\n        barrierState = valueOnCrossing;\n\n## References\n\nThe barrier function expectes exactly one reference: an **underlying**.\n\n### [ref 0]\n\nThe underlying process.\n\n## Arguments\n\n### [arg 0] level\nBarrier level\n\n### [arg 1] valueOnCrossing\nThe value we set when the barrier is crossed.\n\n### [arg 2] direction\n\nThis is the barrier crossing condition.\n\n| value | direction  |\n| ------------- |-------------:|\n| -1 | down          |\n|  0 | any (up or down) |\n| +1 | up |\n    .has_start  = false,\n    .nargs_min  = 4,\n    .nrefs_min  = 1,\nr of time windows\n\nThe number of time

In [7]:
from IPython.display import Markdown as md
doc = ''
for f in functions:
    doc += f"# {f['name']}\n"
    doc += f"## {f['title']}\n"
    doc += f['doc_md'] + '\n\n'
    doc += '---\n\n'
md(doc)

# Barrier
## Barrier (up/down) with (optionally) time windows

# Barrier

A barrier state starts from some initial value S0.
On each computation step we verify the barrier crossing condition, and if it is satisfied, the barrier state will
be set to STATE=valueOnCrossing.

For direction 'up':

    if(Sold<level and Snew>=level)
        barrierState = valueOnCrossing;

For direction 'down':

    if(Sold>level and Snew<=level)
        barrierState = valueOnCrossing;

## References

The barrier function expectes exactly one reference: an **underlying**.

### [ref 0]

The underlying process.

## Arguments

### [arg 0] level
Barrier level

### [arg 1] valueOnCrossing
The value we set when the barrier is crossed.

### [arg 2] direction

This is the barrier crossing condition.

| value | direction  |
| ------------- |-------------:|
| -1 | down          |
|  0 | any (up or down) |
| +1 | up |
    .has_start  = false,
    .nargs_min  = 4,
    .nrefs_min  = 1,
r of time windows

The number of time windows (say **m**) when the barrier will be checked.
The next **m * 2** parameters will be treated as [from,to) pairs, time windows
where the barrier condition will be checked. If **m==0**, the barrier condition
will be checked on all time step.

## Examples

### S0=0, args=[1000,1,-1,0,0]

The barrier inital state is 0. We check on any time step (because arg[4]==0)
the the barrier level==1000 will be crossed in the down direction (for example,
from 1000.5 to 1000). If the condition is NOT fulfilled, we don't change the
barrier state. Otherwise, we set the barrier value (arg[3]==0 means 'set')
to value 1 (because arg[1]==1). Obvioulsy, only the first crossing over
the barrier will have an effect, when we change the barrier state from 0
(because S0==0) to 1 (because arg[1]==1 and arg[3]==0)

### S0=0, args=[1000,1,-1,0,2,111.1,222.2,333.3,444.4]

The same as above, but wee only check the barrier condition if the time
is in one of the two (because arg[5]==2) windows:

- [111.1,222.2)
- [333.3,444.4)

### S0=0, args=[1000,1,-1,1,2,111.1,222.2,333.3,444.4]

The same as above, we 'count' (arg[3]==1 is the increment)
how many times the barrier is crossed in the 'down' direction.


---

# BrownianMotion
## Brownian motion process.

# BrownianMotion

$$dS = \mu dt + \sigma dW_t$$

The function requires _mean_ and _sigma_ parameters and a gaussian random variable to advance its state.
The parameters _mean_ and _sigma_ can be passed either as constant values in _args_ or
they can be themself some processes and in this case the reference process must be provided in _refs_.
A random number can be genegerated internally, or provided externally with _refs_.

## Input parameters

### nrefs=0 and nargs=2

Constant _mean_ and _sigma_, internally generated random variable.

|   | name | range | description |
|---|------|---|---|
| args[0] | mean | any | |
| args[1] | sigma | any | |

### nrefs=1 and nargs=2

Constant _mean_ and _sigma_, externally taken random variable.

|   | name | range | description |
|---|------|---|---|
| refs[0] | W | any | normally distributed variable |
| args[0] | mean | any | |
| args[1] | sigma | any | |

### nrefs=2 and nargs=0

Parameters _mean_ and _sigma_ are given as a reference processes,
the random variable is generated internally.

|   | name | range | description |
|---|------|---|---|
| refs[0] | mean | any | |
| refs[1] | sigma | any | |

### nrefs=3 and nargs=0

Parameters _mean_, _sigma_ and the random variable are given as a reference processes,

|   | name | range | description |
|---|------|---|---|
| refs[0] | mean | any | |
| refs[1] | sigma | any | |
| refs[2] | W | any | |



---

# Division
## Division of two process.

# Division

Compute division of two processes.

If abs(denominator)<=eps, than the division result will be not-a-number.

## Initial state

Not used, the function will override previously computed values.

## Arguments

The function has exactly one arguments. It is a non-negative number eps. 
If denominator state absolute value is less than eps, the division result will be set to NaN.
This can be used to avoid producing +/- infinity values for the division result.
A value 0 can be used (which essentially will allow division by zero).

|   | name | description | range |
|---|------|-------------|-------|
| 0 | eps  | abs(denominator) min value | non-negative |

## References

|   | name |
|---|------|
| 0 | Numerator |
| 1 | Denominator |

## Example:

- start = Ignore
- args = [0.01]
- refs = [2,5]

Take a state[5] value, if abs(state[5])>=0.01 set the division result to state[2]/state[5].
Otherwise, the resukts is NAN.



---

# ExpectedValue
## ExpectedValue

# ExpectedValue

The function will compute $\mathbb{E} S(T)$.

## Input arguments

- refs[0] is the reference process
- args[0] extraction point number (non-negative integer)

### Example

Let's suppose that a model starts simulation at t=0, and we want to evaluate the
model state values at time points t=[10,20,50].
Thus, the model has three extraction points. If we want to use ExpectedValue at
t=20 for a state S=9, we will need
to pass into ExpectedValue function:

- refs[0]=9
- args[0]=1



---

# ForwardZeroCouponBond
## ForwardZeroCouponBond

# ForwardZeroCouponBond

Compute the discount bond P(T0,T1,T2).

refs: size 1, reference to a state which computes ZeroCouponBond
args: size 2. Time points T1,T2



---

# GeometricalBrownianMotion
## GeometricalBrownianMotion process.

# GeometricalBrownianMotion

$$dS = S(\mu dt + \sigma dW_t)$$

The function requires _mean_ and _sigma_ parameters and a gaussian random variable to advance its state.
The parameters _mean_ and _sigma_ can be passed either as constant values in _args_ or
they can be themself some processes and in this case the reference process must be provided in _refs_.
A random number can be genegerated internally, or provided externally with _refs_.

## Input parameters

### nrefs=0 and nargs=2

Constant _mean_ and _sigma_, internally generated random variable.

|   | name | range | description |
|---|------|---|---|
| args[0] | mean | any | |
| args[1] | sigma | any | |

### nrefs=1 and nargs=2

Constant _mean_ and _sigma_, externally taken random variable.

|   | name | range | description |
|---|------|---|---|
| refs[0] | W | any | normally distributed variable |
| args[0] | mean | any | |
| args[1] | sigma | any | |

### nrefs=2 and nargs=0

Parameters _mean_ and _sigma_ are given as a reference processes,
the random variable is generated internally.

|   | name | range | description |
|---|------|---|---|
| refs[0] | mean | any | |
| refs[1] | sigma | any | |

### nrefs=3 and nargs=0

Parameters _mean_, _sigma_ and the random variable are given as a reference processes,

|   | name | range | description |
|---|------|---|---|
| refs[0] | mean | any | |
| refs[1] | sigma | any | |
| refs[2] | W | any | |



---

# HullWhite1FactorModel
## HullWhite1FactorModel

# HullWhite1FactorModel

One Factor Hull-White Model.

$dr(t) = [theta(t)-alpha(t)*r(t)]*dt + sigma(t)*dW(t)$

The updater accepts either:
- 3 arguments and 0 references (for constant alpha,theta,sigma parameters)
OR
- 0 arguments and 3 references (any t-adapted processes of alpha,theta,sigma)

## Input parameters


### Constant parameters, the random variable is internally generated.

|   | name | range | description |
|---|------|---|---|
| nrefs |  | 0 | |
| nargs |  | 3 | |
| args[0] | alpha | any | |
| args[1] | theta | any | |
| args[2] | sigma | any | |

### Constant parameters, with externally given random variable.

|   | name | range | description |
|---|------|---|---|
| nrefs |  | 1 | |
| refs[0] | W     | any | |
| nargs |  | 3 | |
| args[0] | alpha | any | |
| args[1] | theta | any | |
| args[2] | sigma | any | |

### Parameters are given as processes, the random variable is internally generated.

|   | name | range | description |
|---|------|---|---|
| nrefs |  | 3 | |
| refs[0] | alpha(t) | any | |
| refs[1] | theta(t) | any | |
| refs[2] | sigma(t) | any | |
| nargs |  | 0 | |

### Parameters are given as processes, the random variable is externally generated.

|   | name | range | description |
|---|------|---|---|
| nrefs |  | 4 | |
| refs[0] | alpha(t) | any | |
| refs[1] | theta(t) | any | |
| refs[2] | sigma(t) | any | |
| refs[3] | W        | any | |
| nargs |  | 0 | |

        

---

# Linear1DInterpolation
## One dimensional function interpolation.

# Linear1DInterpolation

Perform one demensional interpolation between [Xmin,Xmax].
The number of interpolation points
is deduced from the number of arguments passed to the function.
The first two arguments are Xmin,Xmax. The rest arguments are Y-values on the
regular grid between [Xmin,Xmax]. Thus, the total nummber of arguments
must be at least 4. And the total number of interpolation points is narg-2.

### Example: args=[1,2,3,4,5]
there are three interpolation points (5-2=3):

- Xmin = 1.0
- Xmax = 2.0
- Y(X=1.0) = 3
- Y(X=1.5) = 4
- Y(X=2.0) = 5

The interpolation function argument is passed as 'Xref'. If Xref=-1, then the argument is time.
Otherwise, it is a state number to will be used as an 'x'-argument.

Linear1DInterpolation returns Y(Xmin) if X<Xmin and Y(Xmax) if X>Xmax.

Linear1DInterpolation does not use the old state value.
Thus, the starting process state value is ignored.



---

# Multiplication
## Multiplication of several processes.

# Multiplication

The function computes

$$
A \cdot \prod_{i=1}^n S_i
$$
where A is a constant args[0] and $S_1,S_2,\dots,S_n$ are process states refs[0], ..., refs[n-1].

## Initial state

Not applicable, the function state is computed on each time step.

## Arguments

The function has one arguments.

|   | name |
|---|------|
| args[0] | A, multiplication factor |

## References

Non-zero number of references: nrefs>0.

## Example:

- args = [0.5]
- refs = [0,7,9]

The function will compute: 0.5*state[0]*state[7]*state[9]



---

# Option
## European option.

# European option

## Initial state

The updater has no initial state.

## Arguments

|   | name   | description   |
|---|--------|---------------|
| 0 | strike |               |
| 1 | type   | 0-call, 1-put |

## References

|   | name   | description   |
|---|--------|---------------|
| 0 | underlying | underlying process S |



---

# Polynom
## Polynom.

# Polynom

The updater will compute a polynom of order N.

The number N is equal to the number of arguments passed to the Polynom updater.
The minimal number is 1, and in this case a constant function is defined.
Otherwise it will compute a0+a1*x+a2*x*x+..., where 'x'-is the
polynom argumnt (see below) and a0,a1... are arguments of the
function.

The polynopm argument 'x' is passed as 'reference', thus the updeter expects
strictly one reference and 1...N arguments. If Xref=-1, then the argument is current MC time.
Otherwise, it is a state number to which will be used as an 'x'-argument.

Examples: 
| Polynom | refs | args | Comment |
| ----- | ---- | ---- | ---- |
| P(t) = 44 | [-1] | [44] | A constant function |
| P(t) = 1+2*t^4 | [-1] | [1,0,0,0,2] | use ref=-1 for the time argument |
| P(x) = -x^2 | [123] | [0,0,-1] | assuming that state 123 holds the 'x' argument |




---

# RandomNormal
## Generate a normally distributed random number.

# RandomNormal

The function can do one of the following:
- generate a single normally distributed random variable
- generate two correlated normally distributed random variables

## Initial state

Not applicable, the function has no initial state.

## Arguments & References

| nrefs | nargs | #states | description |
|-------|-------|---------|-------------|
| 0     | 0     | 1       | Generate a single variable. |
| 0     | 1     | 2       | Generate two correlated (correlation coefficient is args[0]) random variables. |
| 1     | 1     | 1       | Takes a random variable from refs[0] and generate a correlated (correlation coefficient is args[0]) random variable. |



---

# Sum
## Weighted sum of several processes.

# Sum

The function computes a weighted sum of several processes: $\sum_{i=1}^n S_i \cdot w_i$



---

# SumOfFutureValues
## SumOfFutureValues

# SumOfFutureValues


## Arguments _args_

an array of (monotnoically increasing) m time points T[i]:  T[0]<T[1]<...<T[m-1]

## References _args_

An array of size 1 with a reference process number N.

## Description

At every extraction point SumOfFutureValues state is computed as

SumOfFutureValues = sum(i = [j..N]) N[i]

where j is the first time point which is not less than t: T[j]>=t (and j==0 or T[j-1]<t). Which means that all past values will not be taken into the sum.

Between every two extraction points Ta,Tb the SumOfFutureValues value is constant.



---

# SwapFixedLeg
## SwapFixedLeg

# SwapFixedLeg

The function computes the sum (T[i]-T[i-1])*N[i],
for i=1..n

## Input parameters

|   | name | range | description |
|---|------|---|---|
| args[0] | T[0] | | |
| args[1] | T[1] | | |
| args[2] | ...  | | |
| refs[0] | notional*df | | |

- time points Ti=[T0,...Tn]   n>=1, the minimal length of Ti vector is two: [T0,T1]
- notional process



---

# SwapFloatingLeg
## SwapFloatingLeg

# SwapFloatingLeg

The reset (or fixing) time points are i=0,1,..n-1.
At this moment the reference process value N[i] is checked, but the cash flow
is only generated at time point i+1.

## Input parameters

To compute the leg value we need to know:
- notional values at points i=1...n, n values in total
- P_df(T_start,T[i]) discount factors at points i=1...n, n values in total
- P_L(T_start,T[i]) factors at points i=0...n, n+1 values in total

L(T_start,T[i],T[i+1]) = (P'(T_start,T[i])/P'(T_start,T[i+1])-1) / (T[i+1]-T[i])

|   | name | range | description |
|---|------|---|---|
| args[0] | T[0] | | |
| args[1] | T[1] | | |
| args[2] | ...  | | |
| refs[0] | P_df(T_start,T[i])*N[i] | notional*df | |
| refs[1] | P_L(T_start,T[i]) | For L() computation |



---

# ZeroCouponBond
## Zero-coupon bond (dsiscount factor)

# ZeroCouponBond

Requires a reference process (the underlying).



---



In [8]:
import mcsdk as sdk
for j in functions:
    f = sdk.UpdaterDoc()
    sdk.UpdaterDoc_from_json(j,f)
    print(f.title)


Barrier (up/down) with (optionally) time windows
Brownian motion process.
Division of two process.
ExpectedValue
ForwardZeroCouponBond
GeometricalBrownianMotion process.
HullWhite1FactorModel
One dimensional function interpolation.
Multiplication of several processes.
European option.
Polynom.
Generate a normally distributed random number.
Weighted sum of several processes.
SumOfFutureValues
SwapFixedLeg
SwapFloatingLeg
Zero-coupon bond (dsiscount factor)
